In [4]:
import os
import rasterio as ras
import numpy as np 
import matplotlib.pyplot as plt

import cv2
from rasterio.windows import Window
from rasterio.enums import Resampling

In [5]:
# Define input and output folders

input_folder = "data/downloaded_tiffs"  # Path to raw TIFF files
output_folder = "data/processed_tiffs"  # Path to save processed files
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

In [8]:


def process_tiff_with_enhancements(tiff_path, tiff_output_path, png_output_path, rgb_output_path):
    """
    Process a single TIFF file: normalize, enhance, save as GeoTIFF, convert to RGB, and visualize.
    """
    # Open the TIFF file
    with ras.open(tiff_path) as src:
        data = src.read(1)  # Read the first band
        profile = src.profile  # Get metadata
        
        # Normalize the data
        normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))
        
        # Apply Gaussian smoothing
        smoothed_data = cv2.GaussianBlur(normalized_data, (5, 5), 0)
        
        # Apply edge detection
        edges = cv2.Canny((smoothed_data * 255).astype(np.uint8), 50, 150)
        
        # Save the normalized data as a new GeoTIFF
        profile.update(dtype='float32')  # Ensure the profile matches the data type
        with ras.open(tiff_output_path, 'w', **profile) as dst:
            dst.write(normalized_data.astype('float32'), 1)
        print(f"Processed GeoTIFF saved: {tiff_output_path}")
        
        # Save the edges as PNG for visualization
        plt.imsave(png_output_path, edges, cmap='gray')
        print(f"Edge-detected PNG saved: {png_output_path}")
        
        # Convert to RGB and save as a new GeoTIFF
        rgb_data = np.stack([normalized_data] * 3, axis=0)  # Create RGB channels
        profile.update(count=3, dtype='float32')  # Update profile for RGB
        with ras.open(rgb_output_path, 'w', **profile) as dst:
            dst.write(rgb_data.astype('float32'))
        print(f"RGB GeoTIFF saved: {rgb_output_path}")

# Process all TIFF files in the folder
for tiff_file in os.listdir(input_folder):
    if tiff_file.endswith(".tiff") or tiff_file.endswith(".tif"):
        input_path = os.path.join(input_folder, tiff_file)
        tiff_output_path = os.path.join(output_folder, f"processed_{tiff_file}")
        png_output_path = os.path.join(output_folder, f"edges_{os.path.splitext(tiff_file)[0]}.png")
        rgb_output_path = os.path.join(output_folder, f"rgb_{tiff_file}")
        process_tiff_with_enhancements(input_path, tiff_output_path, png_output_path, rgb_output_path)


Processed GeoTIFF saved: data/processed_tiffs\processed_dgm1_32_444_5770_1_nw_2023.tif
Edge-detected PNG saved: data/processed_tiffs\edges_dgm1_32_444_5770_1_nw_2023.png
RGB GeoTIFF saved: data/processed_tiffs\rgb_dgm1_32_444_5770_1_nw_2023.tif
Processed GeoTIFF saved: data/processed_tiffs\processed_dgm1_32_444_5771_1_nw_2023.tif
Edge-detected PNG saved: data/processed_tiffs\edges_dgm1_32_444_5771_1_nw_2023.png
RGB GeoTIFF saved: data/processed_tiffs\rgb_dgm1_32_444_5771_1_nw_2023.tif
Processed GeoTIFF saved: data/processed_tiffs\processed_dgm1_32_444_5772_1_nw_2023.tif
Edge-detected PNG saved: data/processed_tiffs\edges_dgm1_32_444_5772_1_nw_2023.png
RGB GeoTIFF saved: data/processed_tiffs\rgb_dgm1_32_444_5772_1_nw_2023.tif
Processed GeoTIFF saved: data/processed_tiffs\processed_dgm1_32_444_5773_1_nw_2023.tif
Edge-detected PNG saved: data/processed_tiffs\edges_dgm1_32_444_5773_1_nw_2023.png
RGB GeoTIFF saved: data/processed_tiffs\rgb_dgm1_32_444_5773_1_nw_2023.tif
Processed GeoTIFF sa